### Rate Limits

- 20 requests every 1 seconds 
- 100 requests every 2 minutes

### First get 10000 match histories.

- First 10 columns are the champions for blue team, red team and the last columns stands for "Blue team won"

In [1]:
import cassiopeia as cass
import getpass
import random

key = getpass.getpass("Enter Riot API key:")

Enter Riot API key:········


In [2]:
cass.set_riot_api_key(key)  # This overrides the value set in your configuration/settings.
cass.set_default_region("KR")

In [3]:
match = cass.get_match(3449425180)

||start_id|end_id|
|----------|------------|------------|
| Jaewoo   | 3490000000 | 3492500000 |
| Jinyoung | 3492500001 | 3495000000 |
| Hwi      | 3495000001 | 3475000000 |
| Q        | 3475000001 | 3500000000 |

In [6]:
import pickle as pkl
import os
import pandas as pd
from cassiopeia.datastores.riotapi.common import APIRequestError
from datapipelines.pipelines import NotFoundError

start_id = 3492500001
end_id = 3495000000
ids = range(start_id, end_id)

dir_name = '{}-{}'.format(start_id, end_id)
os.mkdir(os.path.join('../data/',dir_name))

data = []
user = os.path.realpath('.').split('/')[2]
k=0

In [7]:
def get_lane(participant):
    try:
        lane = participant.lane.name
        if lane =='bot_lane':
            lane = participant.role.name
        return lane
    except AttributeError:
        return 'No info'

In [ ]:
for i in ids :
    dd = []
    match = cass.get_match(i)
    try:
        version = match.version
        queue = match.queue.value
    except APIRequestError:
        print('Your API key has expired.')
        key = getpass.getpass("Enter new Riot API key:")
        cass.set_riot_api_key(key)
        try:
            version = match.version
            queue = match.queue.value
        except:
            continue
    except:
        continue
    if queue in ['RANKED_SOLO_5x5'] and version == '9.1.257.7563':        
        for participant in match.blue_team.participants :
            lane = get_lane(participant)
            champion = participant.champion.name
            rank = participant.summoner.ranks[match.queue]
            tier = ' '.join([rank.tier.name, rank.division.name])
            
            dd.append((lane, participant.champion.name, tier))
            
        for ban in match.blue_team.bans:
            try:
                dd.append(ban.name)
            except AttributeError:
                dd.append('밴없음')
                
        for participant in match.red_team.participants :
            lane = get_lane(participant)
            champion = participant.champion.name
            rank = participant.summoner.ranks[match.queue]
            tier = ' '.join([rank.tier.name, rank.division.name])
            
            dd.append((lane, participant.champion.name, tier))
            
        for j in range(5) :
            try:
                dd.append(match.red_team.bans[j].name)
            except AttributeError:
                dd.append('밴없음')

        dd.append(match.blue_team.win)
        print('Match id {} is valid.'.format(i))
              
        data.append(dd)
    
    if len(data)==10:
        print("Making {}.pkl.".format(k))
        with open(os.path.join('../data/', dir_name, '{}.pkl'.format(k)), 'wb') as f:
            pkl.dump(data, f)
        k+=1
        data = []

if len(data)!=0:
    with open(os.path.join('../data/', dir_name, '{}.pkl'.format(k)), 'wb') as f:
        pkl.dump(data, f)

Making call: https://ddragon.leagueoflegends.com/realms/kr.json
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3492500001
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3492500002
Making call: https://ddragon.leagueoflegends.com/cdn/9.1.1/data/ko_KR/championFull.json
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/YeE2YgdwrGeM9m1b4dH-xPrJiA8ovJEnhJVbRfleQdNLAgU
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/4SkHNVTh2cStL2bo_0sZe9wdfn55Kj-tYL4glvbZ3QEy85I
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/tz140m4nvgh7Z6po6sb-KI6zTW1RCsdD12b3QdAeLqNn9Q
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/teM6qGcPOV1RmcgW6tmtQ5WtjdlaKkMNPgPx9hFM3r0DpQ0
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/xRibK94j_1YUTIa96ioPpYtdDRBv92jr7EDqkNY37jsVKNI
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-su

Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/PeEGwtmR9YVaoqPG4uAvnSG_d-dcXRXMlB--3yn54zqc7U0
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/YFdzvK3lgal4Ydi1VIrjJC61OrB7nPQETUtJi8GUedc3jL0
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/3ZbJvTx7NUxgxEllgaUOcx0o_mPZW_XQb3Pu-t-pgM5zsjM
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/coblpoPev4mCvYPHSaQj0KGEhCydHxty6IoQXDeK0DX_guk
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/DG4NtV4auv2n9WFsHmTJhIuMLKlE-D8ISVImKdWJNtmSOw
Match id 3492500028 is valid.
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3492500029
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3492500030
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/ChYPCimp2bp1Mg3Yvi6DitSE_NXxV0tPCKm9IQCFx_IXKzE
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/b

Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/TuAIju4ZL_dAorjZG29qWHhJGyERtX_nSzj1S4ElKEqHj3Q
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/5W3quKdtqgLmU1j0VuODMoHhEWgkq0r2QGgnsBBJ_UadNsU
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/AJ9hQqQFym_nmNbdC0ONodSNCKcrI0yUS2jYG0OEdxXJeIk
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/S0G8uAromY69N2FN-yFH24eeRbrDTgQWYYYPmoPk8TEhBGo
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/Nj2BfSapB5umM-MArQrqK_7Q6o9RBkYoLr-zMybj57EpfOY
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/Jp_E1_MBRVopVNc9oucgUse754fN0ByJGkqDzIIb4rx95jE
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/u3N03EGtzL27C0RPRvv2iv0mKgCfpUa4DWHJGnquRxT6RQ
Match id 3492500038 is valid.
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3492500039
Making call: ht

Match id 3492500050 is valid.
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3492500051
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3492500052
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/y_aS344THioQG6aIjh7ogUs2izEorrm4h_AMHAfJIixL2qs
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/d-4sE-S4B2P9A4DukltKznDkeDMfputoOMaiixA5xaJbvg
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/0Kq9ErGy3q_BlA3K1r0_jO8javwzecxt689EZvJMXu_5mg
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/e8xpDzwyVqewrdUTHTvfkwNqiTdcTbjgROQMnxYvDXoD0w
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/lYGneRZWx37603EBDkzP7uYg7jnq5qWHjlPH3qGNpSW7Lgg
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/LkmSsTSsVkVCx_SIlWhe_Z7EsDIC5s4L7xZoZPbmeCiHQgg
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-